# Import

In [1]:
# Warning control
import warnings
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

from IPython.display import Markdown
from pydantic import BaseModel
import json
from pprint import pprint

load_dotenv()  # take environment variables from .env.
warnings.filterwarnings("ignore")

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

# Tools

In [2]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Agents

In [3]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue " "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
)

# Agent 2: Logistics Manager
logistics_manager = Agent(
    role="Logistics Manager",
    goal=("Manage all logistics for the event " "including catering and equipmen"),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
)

# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and " "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
)

# Venue (Pydantic Object)

* Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
* Agents will populate this object with information about different venues by creating different instances of it.

In [4]:
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Tasks


* By using `output_json`, you can specify the structure of the output you want.
* By using `output_file`, you can get your output in a file.
* By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [5]:
venue_task = Task(
    description="Find a venue in {event_city} "
    "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    # Outputs the venue details as a JSON file
    agent=venue_coordinator,
)

logistics_task = Task(
    description="Coordinate catering and "
    "equipment for an event "
    "with {expected_participants} participants "
    "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,  # `async_execution=True` means task can run in parallel with tasks that come after it.
    agent=logistics_manager,
)

marketing_task = Task(
    description="Promote the {event_topic} "
    "aiming to engage at least"
    "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,
)

# Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [6]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True,
)

# Running the crew

In [7]:
# event_details = {
#     "event_topic": "Tech Innovation Conference",
#     "event_description": "A gathering of tech innovators "
#     "and industry leaders "
#     "to explore future technologies.",
#     "event_city": "San Francisco",
#     "tentative_date": "2024-09-15",
#     "expected_participants": 500,
#     "budget": 20000,
#     "venue_type": "Conference Hall",
# }

event_details_2 = {
    "event_topic": "Life Insurance: The Future of Underwriting",
    "event_description": "Explore the evolving landscape of life insurance underwriting in where industry leaders, data scientists, "
    "and innovators discuss how artificial intelligence, big data, and customer-centric approaches are transforming the underwriting process."
    "Key topics include AI-driven risk assessment, leveraging health and wellness data, the impact of large language models (like GPT-4) on compliance and customer interaction,"
    "and blockchain’s role in data security. Attendees will gain insights into regulatory challenges, the importance of transparency, and the future skills necessary for a tech-enhanced underwriting workforce.",
    "event_city": "Singapore",
    "tentative_date": "2024-12-01",
    "expected_participants": 500,
    "budget": 20000,
    "venue_type": "Conference Hall",
}

result = event_management_crew.kickoff(inputs=event_details_2)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Singapore that meets criteria for Life Insurance: The Future of Underwriting.


> Entering new CrewAgentExecutor chain...
To find a suitable venue for the event "Life Insurance: The Future of Underwriting" in Singapore, I need to consider the event's theme, size, and budget constraints. I will start by searching the internet for venues in Singapore that could potentially meet these criteria.

Action: Search the internet
Action Input: {"search_query": "venues for conferences in Singapore"} 


Search results: Title: Top 20 Conference Venues Rental in Singapore - Venuerific
Link: https://www.venuerific.com/sg/conference-venues-singapore
Snippet: Discover affordable conference venues for rent in Singapore. Browse photos, compare prices, and book Singapore conference venues with excellent reviews now.
---
Title: The 16 Best Conference Venues for Rent in Singapore - Tagvenue
Link: https://www.tagvenue.co

In [8]:
with open("venue_details.json") as f:
    data = json.load(f)

pprint(data)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
To effectively market the "Life Insurance: The Future of Underwriting" event and engage with at least 500 potential attendees, I need to gather information on marketing strategies and determine how to address the capacity issue since the venue only holds 150 people. 

First, I'll search the internet for effective marketing strategies for events similar to ours. Then, I'll consider how to deal with the limited capacity.

Action: Search the internet
Action Input: {"search_query": "effective marketing strategies for insurance industry events"} 


Search results: Title: 5 Effective Marketing Strategies for Insurance Companies | Sagefrog
Link: https://www.sagefrog.com/blog/marketing-mix-survey/5-effective-marketing-strategies-for-insurance-companies/
Snippet: Consider sponsoring a local sports team or 5K race, volunteering for local organizations, and buying booths at events, such as arts festivals. Thes

{'address': 'Promenade MRT Station, Marina Centre, Singapore',
 'booking_status': 'Available',
 'capacity': 150,
 'name': 'Ocean 1-6, Pan Pacific Singapore'}
